# Text2sql  Evaluate Finetune v1 with Execution Accurrcy

In [1]:
!pip install -q -U torch auto-gptq transformers optimum
!pip install -q -U peft trl einops accelerate xformers bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━

In [2]:
! pip install torchvision==0.20.1 torch==2.5.1 torchaudio==2.5.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0
    Uninstalling torch-2.6.0:
      Successfully uninstalled torch-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.29.post3 requires torch==2.6.0, but you have torch 2.5.1 which is incompatible.


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import re
import json
import sqlite3
import sqlparse
import os
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import GPTQConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer
from time import perf_counter
from rich import print


In [4]:
torch.cuda.is_available()

True

In [5]:
available_memory = torch.cuda.get_device_properties(0).total_memory
available_memory

15828320256

# Load our model from Huggingface and it's Tokenizer 

In [ ]:
model_name = "khalifa1/Text2sql_LLama2_7b_Finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 4 bits – this is slower and less accurate
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_4bit=True,
        device_map="auto",
        use_cache=True,
    )

## Load Spider dataset and it's database 

In [8]:
spider_db_path = "/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/database"
spider_eval_path = "/kaggle/input/yale-universitys-spider-10-nlp-dataset/spider/train_others.json"

with open(spider_eval_path, "r") as f:
    data = json.load(f)
data[0] 

{'db_id': 'geo',
 'query': 'SELECT city_name FROM city WHERE population  =  ( SELECT MAX ( population ) FROM city WHERE state_name  =  "wyoming" ) AND state_name  =  "wyoming";',
 'query_toks': ['SELECT',
  'city_name',
  'FROM',
  'city',
  'WHERE',
  'population',
  '=',
  '(',
  'SELECT',
  'MAX',
  '(',
  'population',
  ')',
  'FROM',
  'city',
  'WHERE',
  'state_name',
  '=',
  '``',
  'wyoming',
  "''",
  ')',
  'AND',
  'state_name',
  '=',
  '``',
  'wyoming',
  "''",
  ';'],
 'query_toks_no_value': ['select',
  'city_name',
  'from',
  'city',
  'where',
  'population',
  '=',
  '(',
  'select',
  'max',
  '(',
  'population',
  ')',
  'from',
  'city',
  'where',
  'state_name',
  '=',
  'value',
  ')',
  'and',
  'state_name',
  '=',
  'value'],
 'question': 'what is the biggest city in wyoming',
 'question_toks': ['what', 'is', 'the', 'biggest', 'city', 'in', 'wyoming'],
 'sql': {'except': None,
  'from': {'conds': [], 'table_units': [['table_unit', 1]]},
  'groupBy': [],

##  Database connection setup

In [9]:
def get_conn(db_id: str):
    db_file = f"{db_id}.sqlite"
    db_path = os.path.join(spider_db_path, db_id, db_file)
    conn = sqlite3.connect(db_path)
    return conn

def get_db_schema(conn) -> str:
    res = conn.execute("SELECT * FROM sqlite_master").fetchall()
    schema = ""
    for d in res:
        if d[-1] is None:
            continue    
        schema += f"{d[-1]}\n\n"
    return schema

## inference function generate the Query from ques and Schema 

In [15]:
def inference(question: str, schema: str) -> str:
    prompt = """### Instruction:
    You are a powerful text-to-SQL model. \
    Your job is to answer questions about a sqlite database. 
    Adhere to these rules:
    - **Deliberately analyze the question and schema**
    - **Use Table Aliases** to prevent ambiguity
    - **Cast numerators as float for ratios**
    - **Use only one column per query**
    You are given a question and context regarding one or more tables.

    You must output the SQL query that bestanswers the question.

    ### Input:
    {question}
    ### Context:
    {schema}
    ### Response:
    ```sql 
    """.format(question=question, schema=schema)


    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=180,
        do_sample=False,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    
    # Post-processing
    postgres_query = outputs[0].split("```sql")[-1].rstrip("```")
    return postgres_to_sqlite(postgres_query)

def postgres_to_sqlite(query: str) -> str:
    substitutions = [
        (r'ilike', 'LIKE'),
        (r'serial\s*$', 'INTEGER PRIMARY KEY AUTOINCREMENT'),
        (r'start\s+with\s+(\d+)', 'CHECK (id >= \\1)'),
    ]
    for pattern, replacement in substitutions:
        query = re.sub(pattern, replacement, query, flags=re.IGNORECASE)
    return query

## Evaluation script

In [11]:
def compare_sql(gold: str, gen: str, conn, is_ordered=False):
    try:
        gold_res = pd.read_sql(gold, conn)
    except Exception as _:
        return 1

    try:
        gen_res = pd.read_sql(gen, conn)
    except Exception as _:
        print("[Gen Fail]", gen)
        return 0

    accuracy = 0
    if (len(gold_res)) == 0:
        return 1 if len(gen_res) == 0 else 0
    
    gold_len = len(gold_res)
    gen_len = len(gen_res)
    for i in range(min(gold_len, gen_len)):
        gold_record = gold_res.values[i]
        
        if not is_ordered:
            try:
                is_match = gold_record in gen_res.values
            except:
                is_match = False
        else:
            is_match = gold_record == gen_res.values[i]
            if (type(is_match) != bool):
                is_match = is_match.all()
                
        if is_match:
            accuracy += 1

    return accuracy / len(gold_res)
 


##  Main Evaluation loop

In [ ]:
NUM_EVAL =100
total_accuracy = 0

for i, h in enumerate(data[:NUM_EVAL]):
    db_id = h['db_id']
    conn = get_conn(db_id)
    schema = get_db_schema(conn)
    question = h['question']
    gen = inference(question, schema)
    gold = h['query']
    
    sub_accuracy = compare_sql(gold, gen, conn)
    total_accuracy += sub_accuracy

    
    print(f"[{i}] ")
    print(f"Question: {question}")
    print(f"Gold SQL: {sqlparse.format(gold, reident=True)}")

final_accuracy = total_accuracy / NUM_EVAL
print(f"Final Execution Accuracy: {final_accuracy * 100:.2f}%")

[0]

Question: where is the ohio river

Gold SQL: SELECT traverse FROM river WHERE river_name  =  "ohio";

[1]

Question: which states does the ohio river run through

...

Gold SQL: SELECT traverse FROM river WHERE LENGTH  =  ( SELECT MAX ( LENGTH ) FROM river );

[99]

Question: what are the states through which the longest river runs

Gold SQL: SELECT traverse FROM river WHERE LENGTH  =  ( SELECT MAX ( LENGTH ) FROM river );

Final Execution Accuracy: 66.67%